## 📘 Preamble: Constructing Composite Economic Indices

This notebook generates composite indices that capture latent dimensions of economic well-being and stress — such as affordability, volatility, mobility constraints, or public anxiety — using publicly available time series from the U.S. Federal Reserve (FRED) and related government sources.

Rather than relying on opaque machine learning models or GPT-generated weights, we’ve grounded these indices in **economic theory and practical policy relevance**. Each composite is defined by:
- **A core purpose** (e.g., to capture fear in public sentiment or stress from rising housing costs)
- **A set of economic components** drawn from trusted FRED indicators
- **A transparent formula** using weighted normalized values, chosen based on:
  - **Thematic relevance**: How directly the indicator reflects the latent concept
  - **Emotional salience**: Whether the indicator captures perception, fear, or immediate reaction (e.g., VIX, sentiment surveys)
  - **Temporal sensitivity**: Whether the indicator reflects fast-moving shifts vs. structural trends
  - **Established economic usage**: When possible, indicators and formulas reflect how economists or policymakers interpret the signals

The result is a set of interpretable, reproducible, and adjustable indices that can be used for analysis, visualization, or downstream modeling.

Each formula is listed explicitly in the table below, with sources and justification. While these formulas are not “official” or derived from econometric regressions, they are informed by research and policy practice — and can evolve as more feedback or data becomes available.

### ✅ Index Construction Pipeline
- All FRED time series are normalized to `[0, 1]` using `MinMaxScaler`
- Composite scores are calculated monthly (`MS`) from January 1997 to present
- Formulas are weighted sums of normalized indicators, with weights based on conceptual strength
- Output is saved as `layman_fred_composites.csv`

> ⚠️ **Note**: These indices are not prescriptive. They are prototypes for public interest analysis and can be iteratively refined or adapted for local, global, or policy-specific use.

### 📊 Composite Index Formulas and Reasoning Table

| **Composite Index**                   | **Components**                                     | **Economic Reasoning & Calculation Principles**                                                                                                                                                                         | **Proposed Formula**                                                                                   | **Sources & Reasoning**                                                                                                                                                           |
|--------------------------------------|---------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Digital Access Index**             | `CUSR0000SERA`                                     | High cable/internet prices reduce access, particularly for low-income households.                                                                                                                                        | `norm(CUSR0000SERA)`                                                                                     | [BLS CPI](https://www.bls.gov/cpi/); affordability as barrier to participation.                                                                                                  |
| **Youth Political Sentiment**        | `LNS14000036`, `LNS11300012`                       | Youth unemployment signals frustration, and low participation suggests disengagement.                                                                                                                                     | `0.6 × norm(LNS14000036) + 0.4 × (1 - norm(LNS11300012))`                                                | [ILO Youth Employment](https://www.ilo.org/global/topics/youth-employment); frustration drives activism/disengagement.                                                           |
| **News Fear Index**                  | `VIXCLS`, `UMCSENT`, `UNRATE`                      | Investor fear (VIX), pessimistic sentiment, and unemployment jointly indicate macro anxiety.                                                                                                                             | `0.5 × norm(VIXCLS) + 0.3 × (1 - norm(UMCSENT)) + 0.2 × norm(UNRATE)`                                    | [CBOE VIX](https://www.cboe.com/tradable_products/vix/); fast-moving fear vs. structural dread.                                                                                    |
| **Crime Sentiment Index**            | `UMCSENT`, `VIXCLS`, `UNRATE`                      | Economic instability and pessimism amplify fear of crime. VIX reflects unease; UNRATE real stress.                                                                                                                       | `0.4 × norm(VIXCLS) + 0.3 × (1 - norm(UMCSENT)) + 0.3 × norm(UNRATE)`                                    | [Becker 1968](https://www.nber.org/books-and-chapters/economics-crime-introduction); unemployment and instability link to fear of crime.                                          |
| **Financial Anxiety Index**          | `BAMLH0A0HYM2EY`, `UMCSENT`, `PSAVERT`             | Credit spread = market fear; sentiment = outlook; savings = cushion. Low savings magnify anxiety.                                                                                                                        | `0.4 × norm(BAMLH0A0HYM2EY) + 0.4 × (1 - norm(UMCSENT)) + 0.2 × (1 - norm(PSAVERT))`                      | [FRED spreads](https://fred.stlouisfed.org); low savings as risk multiplier.                                                                                                      |
| **Recession Concern Index**          | `UMCSENT`, `UNRATE`, `TOTALSL`                     | Pessimism + unemployment + borrowing = looming recession concern.                                                                                                                  | `0.2 × (1 - norm(UMCSENT)) + 0.4 × norm(UNRATE) + 0.4 × norm(TOTALSL)`                                   | [NBER recession indicators](https://www.nber.org/research/data); high leverage + joblessness = red flags.                                                                        |
| **Gas Supply Stress Index**          | `GASREGCOVW`, `VIXCLS`, `RSAFS`                    | Gas price shocks hit consumers first; VIX reflects volatility; sales show behavioral response.                                                                                                                           | `0.5 × norm(GASREGCOVW) + 0.3 × norm(VIXCLS) + 0.2 × (1 - norm(RSAFS))`                                  | [EIA](https://www.eia.gov/petroleum/weekly/); gas prices affect spending elasticity.                                                                                              |
| **School Closure Score**             | `LNS14000036`, `LNS11300012`, `RRVRUSQ156N`        | Youth unemployment + disengagement + housing instability = school closure proxy.                                                                                                                                          | `0.4 × norm(LNS14000036) + 0.3 × (1 - norm(LNS11300012)) + 0.3 × norm(RRVRUSQ156N)`                      | [Brookings](https://www.brookings.edu/research/the-pandemics-effects-on-schools-students); structural dislocation linked to closures.                                             |
| **Retail Activity Index**            | `RSAFS`, `UNRATE`, `PSAVERT`                       | Sales = health, unemployment = constraint, savings = discretionary buffer.                                                                                                                                                | `0.5 × (1 - norm(RSAFS)) + 0.3 × norm(UNRATE) + 0.2 × (1 - norm(PSAVERT))`                               | [Fed](https://www.federalreserve.gov/econres/notes/feds-notes/consumption-and-the-pandemic-20210114.html); sales trends lead perception.                                          |
| **Airfare Affordability Index**      | `CUSR0000SETG01`, `PSAVERT`, `PAYEMS`              | Airfare cost vs. income and savings buffer determines affordability.                                                                                                                                                      | `0.5 × norm(CUSR0000SETG01) + 0.3 × (1 - norm(PSAVERT)) + 0.2 × (1 - norm(PAYEMS))`                       | [BLS](https://www.bls.gov/cpi/), [BLS wages](https://www.bls.gov/ces/); cost vs. disposable income.                                                                               |
| **Rent Stress Index**                | `RRVRUSQ156N`, `CUSR0000SEHA`, `PSAVERT`           | Tight rental markets, high rent, and low savings = housing stress.                                                                                                                                                        | `0.4 × (1 - norm(RRVRUSQ156N)) + 0.4 × norm(CUSR0000SEHA) + 0.2 × (1 - norm(PSAVERT))`                    | [Harvard JCHS](https://www.jchs.harvard.edu/); rent pressure + housing instability = displacement risk.                                                                           |
| **Youth Disconnection Score**        | `LNS14000036`, `LNS11300012`                       | Youth not in labor force or unemployed = detachment.                                                                                                                                                                       | `0.5 × norm(LNS14000036) + 0.5 × (1 - norm(LNS11300012))`                                                | [Measure of America](https://measureofamerica.org/disconnected-youth/); both indicators essential.                                                                                |
| **Household Financial Stress Index** | `PSAVERT`, `TERMCBAUTO48NS`, `TOTALSL`, `MORTGAGE30US` | Low savings, high credit + mortgage rates = compounded stress.                                                                                                                                                             | `0.25 × (1 - norm(PSAVERT)) + 0.25 × norm(TERMCBAUTO48NS) + 0.25 × norm(TOTALSL) + 0.25 × norm(MORTGAGE30US)` | [Fed Stability Report](https://www.federalreserve.gov/publications/financial-stability-report.htm); multi-faceted view of consumer debt load.                                     |
| **Mobility Constraint Index**        | `GASREGCOVW`, `CUSR0000SETG`, `CUSR0000SETG01`     | High costs across gas, public transit, airfare reduce travel freedom.                                                                                                                                                     | `1/3 × (norm(GASREGCOVW) + norm(CUSR0000SETG) + norm(CUSR0000SETG01))`                                   | [BLS CPI](https://www.bls.gov/cpi/); transportation cost = constraint on movement.                                                                                                 |
| **Economic Volatility Score**        | `VIXCLS`, `FEDFUNDS`, `UMCSENT`, `UNRATE`          | Market, monetary, and labor volatility combined. VIX is most reactive, followed by Fed rate changes.                                                                                                                      | `0.4 × norm(VIXCLS) + 0.3 × norm(FEDFUNDS) + 0.2 × (1 - norm(UMCSENT)) + 0.1 × norm(UNRATE)`              | [CBOE](https://www.cboe.com/tradable_products/vix/), [Fed](https://www.federalreserve.gov/); panic and policy shifts drive turbulence.                                             |

In [43]:
# 📦 Imports
from fredapi import Fred
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# 🔧 Setup
fred = Fred(api_key="YOUR API KEY HERE")
scaler = MinMaxScaler()
start_date = "1997-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")
freq = "MS"
date_index = pd.date_range(start=start_date, end=end_date, freq=freq)

# 📥 Pull FRED data
all_series = ["CUSR0000SERA", "LNS14000036", "LNS11300012", "VIXCLS", "UMCSENT", "UNRATE",
              "BAMLH0A0HYM2EY", "PSAVERT", "TOTALSL", "GASREGCOVW", "RSAFS", "RRVRUSQ156N",
              "CUSR0000SETG01", "PAYEMS", "CUSR0000SEHA", "TERMCBAUTO48NS", "MORTGAGE30US",
              "FEDFUNDS", "CUSR0000SETG"]

raw = pd.DataFrame(index=date_index)
for code in all_series:
    raw[code] = fred.get_series(code, observation_start=start_date, observation_end=end_date).resample(freq).ffill()
    print(f"✅ Pulled: {code}")

# 🧼 Normalize data
norm = pd.DataFrame(index=raw.index)
for col in raw.columns:
    norm[col] = scaler.fit_transform(raw[[col]])

# 🧠 Compute composites explicitly
layman_composites = pd.DataFrame(index=date_index)

# Digital Access
layman_composites["layman__digital_access_index"] = 1 - norm["CUSR0000SERA"]

# Youth Political Sentiment
layman_composites["layman__youth_political_sentiment"] = (
    0.6 * norm["LNS14000036"] + 0.4 * (1 - norm["LNS11300012"]))

# News Fear
layman_composites["layman__news_fear_index"] = (
    0.5 * norm["VIXCLS"] + 0.3 * (1 - norm["UMCSENT"]) + 0.2 * norm["UNRATE"])

# Crime Sentiment
layman_composites["layman__crime_sentiment_index"] = (
    0.4 * norm["VIXCLS"] + 0.4 * (1 - norm["UMCSENT"]) + 0.2 * norm["UNRATE"])

# Financial Anxiety
layman_composites["layman__financial_anxiety_index"] = (
    0.5 * norm["BAMLH0A0HYM2EY"] + 0.3 * (1 - norm["UMCSENT"]) + 0.2 * (1 - norm["PSAVERT"]))

# Recession Concern
layman_composites["layman__recession_concern_index"] = (
    0.3 * (1 - norm["UMCSENT"]) + 0.4 * norm["UNRATE"] + 0.3 * norm["TOTALSL"])

# Gas Supply Stress
layman_composites["layman__gas_supply_stress_index"] = (
    0.5 * norm["GASREGCOVW"] + 0.3 * norm["VIXCLS"] + 0.2 * (1 - norm["RSAFS"]))

# School Closure
layman_composites["layman__school_closure_score"] = (
    0.4 * norm["LNS14000036"] + 0.3 * (1 - norm["LNS11300012"]) + 0.3 * (1 - norm["RRVRUSQ156N"]))

# Retail Activity
layman_composites["layman__retail_activity_index"] = (
    0.5 * norm["RSAFS"] + 0.3 * (1 - norm["UNRATE"]) + 0.2 * norm["PSAVERT"])

# Airfare Affordability
layman_composites["layman__airfare_affordability_index"] = (
    0.5 * (1 - norm["CUSR0000SETG01"]) + 0.3 * norm["PAYEMS"] + 0.2 * norm["PSAVERT"])

# Rent Stress
layman_composites["layman__rent_stress_index"] = (
    0.4 * norm["CUSR0000SEHA"] + 0.4 * (1 - norm["RRVRUSQ156N"]) + 0.2 * (1 - norm["PSAVERT"]))

# Youth Disconnection
layman_composites["layman__youth_disconnection_score"] = (
    0.5 * norm["LNS14000036"] + 0.5 * (1 - norm["LNS11300012"]))

# Household Financial Stress
layman_composites["layman__household_financial_stress"] = (
    0.3 * (1 - norm["PSAVERT"]) + 0.2 * norm["TERMCBAUTO48NS"] + 0.3 * norm["TOTALSL"] + 0.2 * norm["MORTGAGE30US"])

# Mobility Constraint
layman_composites["layman__mobility_constraint_index"] = (
    0.5 * norm["GASREGCOVW"] + 0.25 * norm["CUSR0000SETG"] + 0.25 * norm["CUSR0000SETG01"])

# Economic Volatility
layman_composites["layman__economic_volatility_score"] = (
    0.4 * norm["VIXCLS"] + 0.2 * norm["FEDFUNDS"] + 0.2 * (1 - norm["UMCSENT"]) + 0.2 * norm["UNRATE"])

# 🧹 Clean edges
# layman_composites = layman_composites.bfill().ffill()
# 🧹 Smart fill: first forward-fill tail gaps, then backfill head gaps
layman_composites = layman_composites.ffill().bfill()

# Optional: enforce floor/ceiling if normalization ever skews
# layman_composites = layman_composites.clip(lower=0.01, upper=0.99)

# ➕ Add normalized individual indicators for transparency
# for code in all_series:
#     layman_composites[f"fred__{code}"] = norm[code]

layman_composites = layman_composites.mask(layman_composites == 0.0, pd.NA)
# layman_composites = layman_composites.mask(layman_composites == 1.0, pd.NA)

layman_composites = layman_composites.ffill().bfill()

# 💾 Save
layman_composites.index.name = "date"
layman_composites.to_csv("layman_fred_composites.csv")
print("✅ Saved: layman_fred_composites.csv with individual normalized FRED indicators")

# # 💾 Save
# layman_composites.index.name = "date"
# layman_composites.to_csv("layman_fred_composites.csv")
# print("✅ Saved: layman_fred_composites.csv")


✅ Pulled: CUSR0000SERA
✅ Pulled: LNS14000036
✅ Pulled: LNS11300012
✅ Pulled: VIXCLS
✅ Pulled: UMCSENT
✅ Pulled: UNRATE
✅ Pulled: BAMLH0A0HYM2EY
✅ Pulled: PSAVERT
✅ Pulled: TOTALSL
✅ Pulled: GASREGCOVW
✅ Pulled: RSAFS
✅ Pulled: RRVRUSQ156N
✅ Pulled: CUSR0000SETG01
✅ Pulled: PAYEMS
✅ Pulled: CUSR0000SEHA
✅ Pulled: TERMCBAUTO48NS
✅ Pulled: MORTGAGE30US
✅ Pulled: FEDFUNDS
✅ Pulled: CUSR0000SETG
✅ Saved: layman_fred_composites.csv with individual normalized FRED indicators


In [ ]:
#################################
### BELOW CODE IS DEPRECATED  ###
### INCLUDED FOR CODE TRACE / ###
### DEBUGGING PURPOSES ONLY   ###
#################################

In [19]:
# # 📦 Imports
# from fredapi import Fred
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from datetime import datetime

# # 🔧 Setup
# fred = Fred(api_key="YOUR API KEY HERE")
# scaler = MinMaxScaler()
# start_date = "1997-01-01"
# end_date = datetime.today().strftime("%Y-%m-%d")
# freq = "MS"
# date_index = pd.date_range(start=start_date, end=end_date, freq=freq)

# # 🧠 Composite FRED Series Mapping
# # Define composites with validated FRED fallbacks
# composites = {
#     "layman__digital_access_index": [
#         ["CUSR0000SERA"],                 # ✅ Cable and satellite services
#         ["CUUR0000SEEE"],                 # ✅ Telephone services (fallback)
#     ],
#     "layman__youth_political_sentiment": [["LNS14000036", "LNS11300012"]],
#     "layman__news_fear_index": [["VIXCLS", "UMCSENT", "UNRATE"]],
#     "layman__crime_sentiment_index": [["UMCSENT", "VIXCLS", "UNRATE"]],
#     "layman__financial_anxiety_index": [["BAMLH0A0HYM2EY", "UMCSENT", "PSAVERT"]],
#     "layman__recession_concern_index": [["UMCSENT", "UNRATE", "TOTALSL"]],
#     "layman__gas_supply_stress_index": [["GASREGCOVW", "VIXCLS", "RSAFS"]],
#     "layman__school_closure_score": [["LNS14000036", "LNS11300012", "RRVRUSQ156N"]],
#     "layman__retail_activity_index": [["RSAFS", "UNRATE", "PSAVERT"]],
#     "layman__airfare_affordability_index": [["CUSR0000SETG01", "PSAVERT", "PAYEMS"]],
#     "layman__rent_stress_index": [["RRVRUSQ156N", "CUSR0000SEHA", "PSAVERT"]],
#     "layman__youth_disconnection_score": [["LNS14000036", "LNS11300012"]],
#     "layman__household_financial_stress": [["PSAVERT", "TERMCBAUTO48NS", "TOTALSL", "MORTGAGE30US"]],
#     "layman__mobility_constraint_index": [["GASREGCOVW", "CUSR0000SETG", "CUSR0000SETG01"]],
#     "layman__economic_volatility_score": [["VIXCLS", "FEDFUNDS", "UMCSENT", "UNRATE"]],
# }

# # 📥 Pull and normalize data
# all_series = sorted(set(code for groups in composites.values() for group in groups for code in group))
# raw = pd.DataFrame(index=date_index)

# for code in all_series:
#     try:
#         series = fred.get_series(code, observation_start=start_date, observation_end=end_date).resample(freq).ffill()
#         raw[code] = series
#         print(f"✅ Pulled: {code}")
#     except Exception as e:
#         print(f"❌ Failed to pull {code}: {e}")

# # 🧼 Normalize with MinMaxScaler
# norm = pd.DataFrame(index=raw.index)
# for col in raw.columns:
#     norm[col] = scaler.fit_transform(raw[[col]])

# # 🧠 Compute composite layman__ variables
# layman_composites = pd.DataFrame(index=date_index)

# for varname, options in composites.items():
#     for group in options:
#         if all(col in norm.columns for col in group):
#             layman_composites[varname] = norm[group].mean(axis=1).clip(0.01, 0.99)
#             print(f"🧠 Calculated {varname} using: {group}")
#             break
#     else:
#         print(f"❌ Could not calculate {varname} — no valid group found.")

# # 🧹 Fill edge gaps
# layman_composites = layman_composites.bfill().ffill()

# # 💾 Save output
# layman_composites.index.name = "date"
# layman_composites.to_csv("layman_fred_composites.csv")
# print("✅ Saved: layman_fred_composites.csv")

In [9]:
# # 📦 Imports
# from fredapi import Fred
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from datetime import datetime

# # 🔧 Setup
# fred = Fred(api_key="YOUR API KEY HERE")
# scaler = MinMaxScaler()
# start_date = "1997-01-01"
# end_date = datetime.today().strftime("%Y-%m-%d")
# freq = "MS"
# date_index = pd.date_range(start=start_date, end=end_date, freq=freq)

# # 🧠 Composite FRED Series Mapping + Descriptions
# composites = {
#     "layman__digital_access_index": (
#         [["CUSR0000SERA"], ["CUUR0000SEEE"]],
#         "Affordability of digital connectivity (TV, cable, internet, telecom)"
#     ),
#     "layman__youth_political_sentiment": (
#         [["LNS14000036", "LNS11300012"]],
#         "Youth engagement and frustration measured by unemployment and labor participation"
#     ),
#     "layman__news_fear_index": (
#         [["VIXCLS", "UMCSENT", "UNRATE"]],
#         "Public fear via market volatility, consumer sentiment, and job security"
#     ),
#     "layman__crime_sentiment_index": (
#         [["UMCSENT", "VIXCLS", "UNRATE"]],
#         "Public sense of instability based on perception of economic and job risk"
#     ),
#     "layman__financial_anxiety_index": (
#         [["BAMLH0A0HYM2EY", "UMCSENT", "PSAVERT"]],
#         "Personal financial stress through credit spread, sentiment, and savings"
#     ),
#     "layman__recession_concern_index": (
#         [["UMCSENT", "UNRATE", "TOTALSL"]],
#         "General worry about recession using sentiment, joblessness, and lending"
#     ),
#     "layman__gas_supply_stress_index": (
#         [["GASREGCOVW", "VIXCLS", "RSAFS"]],
#         "Concerns about gas prices and availability (real and speculative)"
#     ),
#     "layman__school_closure_score": (
#         [["LNS14000036", "LNS11300012", "RRVRUSQ156N"]],
#         "Proxy for youth disruption through employment and housing"
#     ),
#     "layman__retail_activity_index": (
#         [["RSAFS", "UNRATE", "PSAVERT"]],
#         "In-person consumer behavior and confidence"
#     ),
#     "layman__airfare_affordability_index": (
#         [["CUSR0000SETG01", "PSAVERT", "PAYEMS"]],
#         "Affordability of flying based on income, airfare, and savings"
#     ),
#     "layman__rent_stress_index": (
#         [["RRVRUSQ156N", "CUSR0000SEHA", "PSAVERT"]],
#         "Housing pressure based on rent vacancy, cost, and savings"
#     ),
#     "layman__youth_disconnection_score": (
#         [["LNS14000036", "LNS11300012"]],
#         "Youth neither in school nor working"
#     ),
#     "layman__household_financial_stress": (
#         [["PSAVERT", "TERMCBAUTO48NS", "TOTALSL", "MORTGAGE30US"]],
#         "Stress from auto loans, housing, credit, and savings"
#     ),
#     "layman__mobility_constraint_index": (
#         [["GASREGCOVW", "CUSR0000SETG", "CUSR0000SETG01"]],
#         "Burden of transportation from gas, transit, and airfare costs"
#     ),
#     "layman__economic_volatility_score": (
#         [["VIXCLS", "FEDFUNDS", "UMCSENT", "UNRATE"]],
#         "Macro instability due to markets, policy, and jobs"
#     ),
# }

# # 📥 Load weights from CSV
# weights_df = pd.read_csv("layman_gpt_weights_parsed.csv")

# # 📥 Pull FRED data
# all_series = sorted(set(code for (groups, _) in composites.values() for group in groups for code in group))
# raw = pd.DataFrame(index=date_index)

# for code in all_series:
#     try:
#         series = fred.get_series(code, observation_start=start_date, observation_end=end_date).resample(freq).ffill()
#         raw[code] = series
#         print(f"✅ Pulled: {code}")
#     except Exception as e:
#         print(f"❌ Failed to pull {code}: {e}")

# # 🧼 Normalize data
# norm = pd.DataFrame(index=raw.index)
# for col in raw.columns:
#     norm[col] = scaler.fit_transform(raw[[col]])

# # 🧠 Compute composites using positional weights
# layman_composites = pd.DataFrame(index=date_index)

# for varname, (group_list, desc) in composites.items():
#     for group in group_list:
#         if all(code in norm.columns for code in group):
#             try:
#                 weights_subset = weights_df[weights_df["variable"] == varname]
#                 weights = [weights_subset[weights_subset["component_idx"] == i]["weight"].values[0]
#                            for i in range(len(group))]
#                 layman_composites[varname] = norm[group].dot(weights).clip(0.01, 0.99)
#                 print(f"🧠 Calculated {varname} with weights: {weights}")
#                 break
#             except Exception as e:
#                 print(f"⚠️ Failed {varname} — fallback to equal weights: {e}")
#                 weights = [1 / len(group)] * len(group)
#                 layman_composites[varname] = norm[group].dot(weights).clip(0.01, 0.99)
#                 print(f"➡️ Used fallback weights: {weights}")
#         else:
#             print(f"❌ Skipped {varname} — missing inputs")

# # 🧹 Clean edges
# layman_composites = layman_composites.bfill().ffill()

# # 💾 Save
# layman_composites.index.name = "date"
# layman_composites.to_csv("layman_fred_composites.csv")
# print("✅ Saved: layman_fred_composites.csv")

✅ Pulled: BAMLH0A0HYM2EY
✅ Pulled: CUSR0000SEHA
✅ Pulled: CUSR0000SERA
✅ Pulled: CUSR0000SETG
✅ Pulled: CUSR0000SETG01
✅ Pulled: CUUR0000SEEE
✅ Pulled: FEDFUNDS
✅ Pulled: GASREGCOVW
✅ Pulled: LNS11300012
✅ Pulled: LNS14000036
✅ Pulled: MORTGAGE30US
✅ Pulled: PAYEMS
✅ Pulled: PSAVERT
✅ Pulled: RRVRUSQ156N
✅ Pulled: RSAFS
✅ Pulled: TERMCBAUTO48NS
✅ Pulled: TOTALSL
✅ Pulled: UMCSENT
✅ Pulled: UNRATE
✅ Pulled: VIXCLS
🧠 Calculated layman__digital_access_index with weights: [1.0]
🧠 Calculated layman__youth_political_sentiment with weights: [0.5, 0.5]
🧠 Calculated layman__news_fear_index with weights: [0.4, 0.3, 0.3]
🧠 Calculated layman__crime_sentiment_index with weights: [0.3, 0.35, 0.35]
🧠 Calculated layman__financial_anxiety_index with weights: [0.4, 0.3, 0.3]
🧠 Calculated layman__recession_concern_index with weights: [0.3, 0.35, 0.35]
🧠 Calculated layman__gas_supply_stress_index with weights: [0.4, 0.3, 0.3]
🧠 Calculated layman__school_closure_score with weights: [0.35, 0.35, 0.3]
🧠 Calc